In [ ]:
from dotenv import load_dotenv
load_dotenv()

import pprint
pp = pprint.PrettyPrinter(indent=4)

True

## Loading model

In [8]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("llama3-8b-8192", model_provider="groq")

### News api response

In [ ]:
# code from news_search_tool.py

import requests
import os
query = 'IPL 2025'
url = "https://newsdata.io/api/1/news"
params = {
    "q": query,
    "language": "en",
    "size": 3,
    "apikey":os.getenv("NEWS_API_KEY")
}
response = requests.get(url, params=params)
res = response.json()

In [42]:
res['results'][0]['description']

'Gujarat Titans skipper Shubman Gill played a mature knock on Sunday as he scored 61* off 43 balls as his side reached a 153-run target set by SunRisers Hyderabad in just 16.3 overs'

In [33]:
from news_search_tool import search_news

news_data = search_news('IPL 2025')

In [41]:
pp.pprint(news_data)

('Mohammed Siraj: From Doubt to Dominance in IPL 2025 - Mohammed Siraj: From '
 'Doubt to Dominance in IPL 2025 In the ongoing edition of the Indian Premier '
 'League (IPL) 2025, former cricketer-turned-commentator Navjot Singh Sidhu '
 'has heaped praise on Mohammed Siraj, the pace spearhead for Gujarat Titans. '
 "Siraj has been instrumental in quelling Sunrisers Hyderabad's formidable "
 'batting lineup with his impressive swing bowling.Having faced a tumultuous '
 "phase after being dropped from India's Champions Trophy squad due to "
 'ineffective performances in crucial overs, Siraj has emerged with renewed '
 'vigor. His quest to reclaim his position in the national team saw him '
 "excelling in IPL, rattling opponents' top orders with his tenacity and "
 'skill.Beginning the season with a shaky performance against Punjab Kings, '
 'where he gave away 54 runs, Siraj transformed rapidly. His remarkable '
 "comeback included dismantling Mumbai Indians' openers and playing a decis

## Prompt template
#### "fernandoruiz/information_extraction_for_summarization"

In [ ]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langchain import hub

prompt = hub.pull("fernandoruiz/information_extraction_for_summarization")

# prompt template
'''
Please identify if the following news article is related to this topic : {topic}.
    [Start news article]:
    {article}
    [End news article]
    If the article is related to the topic, return a summary of the article related to the given topic.
    Answer in the most factual way possible. Only use content from the article.
    Please return the result in json using the following keys :
    "relevant"
    "who"
    "what"
    "where"
    "when"
    "why"
    If the article is not related return : "relevant" : false
    Answer:
'''

c:\Users\Admin\anaconda3\envs\langenv\lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# injects the 'topic' and 'article' into the prompt
injected_prompt = prompt.invoke({"topic": "IPL 2025", "article": news_data})

# invokes the model with the injected prompt
answer = llm.invoke(injected_prompt)
answer

AIMessage(content='Here is the JSON response:\n\n{\n"relevant": true,\n"who": "Mohammed Siraj",\n"What": "dominance in IPL 2025",\n"where": "Gujarat Titans, Sunrisers Hyderabad, and Wankhede Stadium",\n"When": "Ongoing edition of IPL 2025",\n"Why": "Siraj\'s impressive swing bowling and tenacity",\n"Extra info": "Siraj was praised by Navjot Singh Sidhu and has been instrumental in quelling Sunrisers Hyderabad\'s batting lineup."\n}\n\nNote: The article is related to the topic "IPL 2025" as it discusses Mohammed Siraj\'s performance in the ongoing edition of the Indian Premier League (IPL) 2025.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 547, 'total_tokens': 699, 'completion_time': 0.126666667, 'prompt_time': 0.068811847, 'queue_time': 0.16215481799999998, 'total_time': 0.195478514}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-d0392321-e3cf-40c9-b5e9-

In [45]:
pp.pprint(injected_prompt.text)

('Please identify if the following news article is related to this topic : IPL '
 '2025.\n'
 '    [Start news article]:\n'
 '    Mohammed Siraj: From Doubt to Dominance in IPL 2025 - Mohammed Siraj: '
 'From Doubt to Dominance in IPL 2025 In the ongoing edition of the Indian '
 'Premier League (IPL) 2025, former cricketer-turned-commentator Navjot Singh '
 'Sidhu has heaped praise on Mohammed Siraj, the pace spearhead for Gujarat '
 "Titans. Siraj has been instrumental in quelling Sunrisers Hyderabad's "
 'formidable batting lineup with his impressive swing bowling.Having faced a '
 "tumultuous phase after being dropped from India's Champions Trophy squad due "
 'to ineffective performances in crucial overs, Siraj has emerged with renewed '
 'vigor. His quest to reclaim his position in the national team saw him '
 "excelling in IPL, rattling opponents' top orders with his tenacity and "
 'skill.Beginning the season with a shaky performance against Punjab Kings, '
 'where he gave away 5

In [ ]:
pp.pprint(answer.content)

('Here is the JSON response:\n'
 '\n'
 '{\n'
 '"relevant": true,\n'
 '"who": "Mohammed Siraj",\n'
 '"What": "dominance in IPL 2025",\n'
 '"where": "Gujarat Titans, Sunrisers Hyderabad, and Wankhede Stadium",\n'
 '"When": "Ongoing edition of IPL 2025",\n'
 '"Why": "Siraj\'s impressive swing bowling and tenacity",\n'
 '"Extra info": "Siraj was praised by Navjot Singh Sidhu and has been '
 'instrumental in quelling Sunrisers Hyderabad\'s batting lineup."\n'
 '}\n'
 '\n'
 'Note: The article is related to the topic "IPL 2025" as it discusses '
 "Mohammed Siraj's performance in the ongoing edition of the Indian Premier "
 'League (IPL) 2025.')
